# Importing Libraries

In [43]:
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama
from ollama import chat

# Website class to scrape website data

In [44]:
class Website:
    """
    A utility class to represent a Website that we have scraped
    """

    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [45]:
commerce = Website(
    "https://learn.microsoft.com/en-us/dynamics365/commerce/channel-setup-retail"
)
print(commerce.title)

Set up a retail  channel - Commerce | Dynamics 365 | Microsoft Learn


# System prompt for the model

In [46]:
system_prompt = """
You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related.
Respond in markdown.
"""

# Function to create user prompt for the model

def user_prompt_for(Website):
    user_prompt = f"the Website title is {Website.title}"
    user_prompt += """
    The contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n
    """
    user_prompt += Website.text
    return user_prompt

In [47]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled '{website.title}'.\n"
    user_prompt += "The contents of this website are as follows: Please provide a short summary of this website in markdown. "
    user_prompt += "If it includes news or announcements, summarize these as well.\n\n"
    user_prompt += website.text
    return user_prompt

# Function to summarize the content of a website

In [48]:
def summarize(url):
    website = Website(url)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)},
    ]
    # Disable streaming, get the entire response in one go
    response = ollama.chat(
        model="chevalblanc/gpt-4o-mini", messages=messages, stream=False
    )
    return response["message"]["content"]

# Function to display the summary of a website

In [49]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

# Example usage

In [50]:
display_summary(
    "https://learn.microsoft.com/en-us/dynamics365/commerce/dev-itpro/store-commerce"
)

How can I deactivate the Store Commerce app on a device? 

To deactivate the Store Commerce app, open PowerShell in administrator mode and navigate to the folder where you downloaded the Store Commerce installer executable. In PowerShell, find the Store Commerce installer executable and pass the `StoreCommerce.Installer.exe uninstall` parameter to uninstall the app.

Note: The Store Commerce app shouldn't be run with elevated privileges, and shouldn't be run from an account with elevated privileges.